# Live Sensor Comparison

This notebook serves as a quick way to compare measurements at multiple sensors side-by-side.

In [80]:
import requests
import pandas as pd
from ipywidgets import Dropdown
import ipywidgets as widgets

def fetch_data(location_id: str, start_date: str, end_date: str) -> pd.DataFrame:
  sample_noise_url = f"https://api.tracket.info/v1/locations/{location_id}/noise?granularity=raw&start={start_date}&end={end_date}"
  print(f'Requesting from {sample_noise_url}')

  page_num = 0

  print(f"Requesting at page {page_num}...", end=" ")
  noise_data = requests.get(sample_noise_url, params={'page': page_num}).json()
  noise_data = noise_data['measurements']
  print(f"Fetched {len(noise_data)} rows")


  all_noise_data = noise_data
  while len(noise_data) > 0:
    page_num += 1

    print(f"Requesting at page {page_num}...", end=" ")
    noise_data = requests.get(sample_noise_url, params={'page': page_num}).json()['measurements']
    print(f"Fetched {len(noise_data)} rows")

    if len(noise_data) > 0:
      all_noise_data.extend(noise_data)

  all_noise_data = pd.DataFrame(all_noise_data).set_index('timestamp')

  return all_noise_data

We load the locations first and set up our inputs for the comparison.

In [91]:
location_url = "https://api.tracket.info/v1/locations"

locations = requests.get(location_url).json()
locations = pd.DataFrame(locations['locations'])

text_locations = (locations['id'].astype('str') + " " + locations['label']).to_list()

In [109]:
location_multi_select = widgets.SelectMultiple(
    options=list(zip(text_locations, locations['id'])),
    value=[1086570, 859525],
    description='Locations',
    disabled=False
)

start_date_select = widgets.DatePicker(
    description='Start Date',
    value=(pd.to_datetime('today') - pd.Timedelta('7 days')).date(),
    disabled=False
)

end_date_select = widgets.DatePicker(
    description='End Date',
    value=pd.to_datetime('today').date(),
    disabled=False
)

column_to_show = widgets.Dropdown(
    options=['min', 'max', 'mean'],
    value='min',
    description='Column:',
    disabled=False,
)

Make the selection below for which locations and timeframe to compare:

In [121]:
display(location_multi_select)
display(start_date_select)
display(end_date_select)

SelectMultiple(description='Locations', index=(13, 17), options=(('572227 Scott St. & Wellington St.', 572227)…

DatePicker(value=datetime.date(2025, 5, 24), description='Start Date')

DatePicker(value=datetime.date(2025, 5, 31), description='End Date')

In [130]:
start_date = start_date_select.value
end_date = end_date_select.value

collected_data = {}
for location_id in location_multi_select.value:
  print(f"Fetching data for location {location_id}")
  collected_data[location_id] = fetch_data(location_id, start_date, end_date)

Fetching data for location 859525
Requesting from https://api.tracket.info/v1/locations/859525/noise?granularity=raw&start=2025-05-12&end=2025-05-31
Requesting at page 0... Fetched 1000 rows
Requesting at page 1... Fetched 1000 rows
Requesting at page 2... Fetched 1000 rows
Requesting at page 3... Fetched 1000 rows
Requesting at page 4... Fetched 1000 rows
Requesting at page 5... Fetched 438 rows
Requesting at page 6... Fetched 0 rows
Fetching data for location 1086619
Requesting from https://api.tracket.info/v1/locations/1086619/noise?granularity=raw&start=2025-05-12&end=2025-05-31
Requesting at page 0... Fetched 1000 rows
Requesting at page 1... Fetched 1000 rows
Requesting at page 2... Fetched 1000 rows
Requesting at page 3... Fetched 1000 rows
Requesting at page 4... Fetched 1000 rows
Requesting at page 5... Fetched 473 rows
Requesting at page 6... Fetched 0 rows


Select the column to compare:

In [123]:
display(column_to_show)

Dropdown(description='Column:', index=2, options=('min', 'max', 'mean'), value='mean')

In [131]:
from plotly import graph_objects as go

fig = go.Figure()

for location_id, noise_data in collected_data.items():
  fig.add_trace(go.Scatter(
    x=noise_data.index,
    y=noise_data[column_to_show.value],
    name=str(location_id),
  ))

fig.update_layout(
    title=f"Noise Comparison - {column_to_show.value}",
    xaxis_title="Date",
    yaxis_title="dBA"
  )
fig.show()